In [ ]:
import pandas as pd
from baseline_scores import hiat_score, thrive_score, span100_score
from sklearn import metrics
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
import os
import numpy as np

from prediction.mrs_outcome_prediction.baseline_models.evaluation_helper_functions import evaluate_method

In [ ]:
stroke_registry_path = '/Users/jk1/OneDrive - unige.ch/stroke_research/geneva_stroke_unit_dataset/data/stroke_registry/post_hoc_modified/stroke_registry_post_hoc_modified.xlsx'

In [ ]:
data_df = pd.read_excel(stroke_registry_path)

In [ ]:
data_df

In [ ]:
# only keep data with '3M mRs' not nan
data_df = data_df[data_df['3M mRS'].notnull()]

In [ ]:
data_df['3M mRS 0-1'] = np.where(data_df['3M mRS'].isna(), np.nan, np.where(data_df['3M mRS'] <= 1, 1, 0))
data_df['3M mRS 0-2'] = np.where(data_df['3M mRS'].isna(), np.nan, np.where(data_df['3M mRS'] <= 2, 1, 0))

In [ ]:
# mrs forwarding (model simply outputs premorbid mrs)
data_df['mrs01_forwarding good outcome pred'] = data_df['Prestroke disability (Rankin)'] <= 1
data_df['mrs01_forwarding_prob'] = data_df['Prestroke disability (Rankin)'] <= 1
data_df['mrs02_forwarding good outcome pred'] = data_df['Prestroke disability (Rankin)'] <= 2
data_df['mrs02_forwarding_prob'] = data_df['Prestroke disability (Rankin)'] <= 2

In [ ]:
data_df['HIAT_prob'] =  data_df.apply(
    lambda subject: hiat_score(
        subject['Age (calc.)'],
        subject['NIH on admission'],
        subject['1st glucose']),
    axis=1)

# defined as mRS < 4 at discharge
data_df['HIAT good outcome pred'] = data_df['HIAT_prob'] > 0.5

In [ ]:
data_df['span100_prob'] =  data_df.apply(
    lambda subject: span100_score(
        subject['Age (calc.)'],
        subject['NIH on admission']),
    axis=1)
data_df['span100 good outcome pred'] = data_df['span100_prob'] > 0.5

In [ ]:
data_df['THRIVE_prob'] = data_df.apply(
    lambda subject: thrive_score(
        subject['Age (calc.)'],
        subject['NIH on admission'],
        subject['MedHist Hypertension'],
        subject['MedHist Diabetes'],
        subject['MedHist Atrial Fibr.']
    ),
    axis=1)

data_df['THRIVE good outcome pred'] = data_df['THRIVE_prob'] > 0.5

In [ ]:
from prediction.mrs_outcome_prediction.baseline_models.baseline_scores import thriveC_score

data_df['THRIVEC_prob'] = data_df.apply(
    lambda subject: thriveC_score(
        subject['Age (calc.)'],
        subject['NIH on admission'],
        subject['MedHist Hypertension'],
        subject['MedHist Diabetes'],
        subject['MedHist Atrial Fibr.']
    ),
    axis=1)

data_df['THRIVEC good outcome pred'] = data_df['THRIVEC_prob'] > 0.5

In [ ]:
data_df[['Age (calc.)',
        'NIH on admission',
        'MedHist Hypertension',
        'MedHist Diabetes',
        'MedHist Atrial Fibr.', 'THRIVE_prob', 'THRIVE good outcome pred', 'THRIVEC_prob', 'THRIVEC good outcome pred']]

In [ ]:
thrive_df = evaluate_method('THRIVE', data_df, ground_truth='3M mRS 0-2')
thriveC_df = evaluate_method('THRIVEC', data_df, ground_truth='3M mRS 0-2')
hiat_df = evaluate_method('HIAT', data_df, ground_truth='3M mRS 0-2')
span100_df = evaluate_method('span100', data_df, ground_truth='3M mRS 0-2')
mrs02_forwarding_df = evaluate_method('mrs02_forwarding', data_df, ground_truth='3M mRS 0-2')

mrs02_result_df = pd.concat([thrive_df, thriveC_df, hiat_df, span100_df, mrs02_forwarding_df])
mrs02_result_df

In [ ]:
thrive_df = evaluate_method('THRIVE', data_df, ground_truth='3M mRS 0-1')
thriveC_df = evaluate_method('THRIVEC', data_df, ground_truth='3M mRS 0-1')
hiat_df = evaluate_method('HIAT', data_df, ground_truth='3M mRS 0-1')
span100_df = evaluate_method('span100', data_df, ground_truth='3M mRS 0-1')
mrs01_forwarding_df = evaluate_method('mrs01_forwarding', data_df, ground_truth='3M mRS 0-1')

mrs01_result_df = pd.concat([thrive_df, thriveC_df, hiat_df, span100_df, mrs01_forwarding_df])
mrs01_result_df

In [ ]:
output_dir = '/Users/jk1/Downloads'
mrs02_result_df.to_csv(os.path.join(output_dir, 'mrs02_clinical_scores_results.csv'))
mrs01_result_df.to_csv(os.path.join(output_dir, 'mrs01_clinical_scores_results.csv'))